In [ ]:
import os
import numpy as np
from PIL import Image
from scipy import ndimage, stats
from scipy.fft import fft, fftfreq, fftn
import skimage.io
from skimage.measure import regionprops, label, find_contours
from skimage.segmentation import flood
import matplotlib.pyplot as plt
from scipy.ndimage import shift, distance_transform_edt, rotate, map_coordinates, zoom
import tifffile as tiff
import cv2
import glob
from scipy.signal import wiener, find_peaks
from circle_fit import taubinSVD, plot_data_circle, hyperSVD, prattSVD, hyperLSQ, lm, standardLSQ

In [ ]:
save_dir = "E:/surgele/mire_2507"

In [ ]:
file_path0=os.path.join(save_dir,'ref.tiff')  
bg_path=os.path.join(save_dir,'bg.tiff')  
# file_path=os.path.join(save_dir,'0.tiff')       
file_pathS = sorted(glob.glob(os.path.join(save_dir,'depths/*.tiff')))

In [ ]:
def pre_processing(angle, pitch, file_path, bg_path=None):
   LFM1 = skimage.io.imread(file_path)
   if bg_path != None:
       bg = skimage.io.imread(bg_path)
       LFM1 = (LFM1 - bg)
   img_rotated = ndimage.rotate(LFM1, angle, reshape=True) 
   LFM1 = img_rotated.astype(np.int32)
   h, w = LFM1.shape[:2]
   fraction1 = 15/pitch
   fraction2 = 15/pitch
   img_resized = np.array(Image.fromarray(LFM1, mode='I').resize((int(fraction1 * w), int(fraction2 * h)), Image.Resampling.BILINEAR))
   return img_resized

# Number of microlenses in X and Y on the raw image (140 x 140 for us)
Nx=140
Ny=140

def calibrate_LFM3(img, x, y):
    step = 15  # espacement entre microlentilles

    LFM = np.zeros((Nx, Ny, step, step))
    XX = np.zeros((Nx, Ny))
    YY = np.zeros((Nx, Ny))
    CenterX = np.zeros((Nx, Ny))
    CenterY = np.zeros((Nx, Ny))
    CenterX_ini = np.zeros((Nx, Ny))
    CenterY_ini = np.zeros((Nx, Ny))

    for ii in range(Nx):
        for jj in range(Ny):

            if ii == 0 and jj == 0:
                cx, cy = x, y
            elif ii == 0:
                # première ligne : moyenne impossible, on prend celle de gauche
                cx = CenterX[ii, jj - 1] + step
                cy = CenterY[ii, jj - 1]
            elif jj == 0:
                # premiere colonne : moyenne impossible, se base sur la lentille au-dessus
                cx = CenterX[ii - 1, jj]
                cy = CenterY[ii - 1, jj] + step
            else:
                # moyenne entre la lentille de gauche et celle du dessus
                cx = (CenterX[ii - 1, jj] + CenterX[ii, jj - 1] + step) / 2
                cy = (CenterY[ii - 1, jj] + CenterY[ii, jj - 1] + step) / 2

            cx = int(cx)
            cy = int(cy)

            # extraction de la microlentille dans l'image
            intermediate = img[cy-(step//2):cy+(step//2 + 1), cx-(step//2):cx+(step//2 + 1)]
            intermediate = intermediate.astype(float)
            
            max_pos = np.unravel_index(np.argmax(intermediate), intermediate.shape)
            thresh = np.percentile(intermediate, 30)
            flooded = flood(intermediate, seed_point=max_pos, tolerance=intermediate[max_pos]-thresh)

            ##### Contours + Circle-fit #####
            flooded = flooded.astype(np.uint8)
            contours, _ = cv2.findContours(flooded, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            main_contour = min(contours, key=len)
            points = main_contour[:, 0, :]  # de (n,1,2) vers (n,2)
            fit = taubinSVD(points)
            ymax = round(fit[1])
            xmax = round(fit[0])
            
            dx = xmax - (step // 2)
            dy = ymax - (step // 2)

            XX[ii, jj] = dx
            YY[ii, jj] = dy

            # ajustement du centre
            cx_adj = cx + dx
            cy_adj = cy + dy
            CenterX_ini[ii, jj] = cx
            CenterY_ini[ii, jj] = cy
            CenterX[ii, jj] = cx_adj
            CenterY[ii, jj] = cy_adj

            # Ré-extraction de la micro-image ajustée
            LFM[ii, jj] = img[cy_adj-(step//2):cy_adj+(step//2 + 1), cx_adj-(step//2):cx_adj+(step//2 + 1)]

    return LFM, CenterX, CenterY, XX, YY, CenterX_ini, CenterY_ini

def get_max_distance_center2(flooded):
    labeled = label(flooded)
    props = regionprops(labeled)
    best_center = None
    best_score = -1
    for region in props:
        minr, minc, maxr, maxc = region.bbox
        if minr == 0 or minc == 0 or maxr == flooded.shape[0] or maxc == flooded.shape[1]:
            continue  # ignore si ca touche le bord de la sous image
        mask = (labeled == region.label)
        dist = distance_transform_edt(mask)
        max_idx = np.unravel_index(np.argmax(dist), dist.shape)
        score = dist[max_idx]
        if score > best_score:
            best_score = score
            best_center = max_idx
    return best_center if best_center is not None else (ndimage.center_of_mass(flooded))

def compute_LFM(img, CenterX, CenterY, XX, YY):
    step = 15
    LFM = np.zeros([Nx, Ny, 15, 15])
    for ii in range(0, Nx):
        for jj in range(0, Ny):    
            LFM[ii, jj, :, :] = img[int(CenterY[ii, jj])-(step//2):int(CenterY[ii, jj])+(step//2 + 1), 
                                     int(CenterX[ii, jj])-(step//2):int(CenterX[ii, jj])+(step//2 + 1)]   
    return LFM

def rendered_focus(C,rendered_height, rendered_width, side, radiance, returnVal=False):
    rendered = np.zeros((rendered_height, rendered_width))
    center_uv = int(side/2)
    for u in range(0, side):
        for v in range(0, side):
            shift_x, shift_y = C*(center_uv-u), C*(center_uv-v)
            rendered[:, :] += shift(radiance[:, :, u, v], (shift_x, shift_y))
    final = rendered / (side*side)
    if returnVal:
        return final

In [ ]:
angle = -0.1099 # -0.057 pour les tests de mai
pitch = 14.02 
x0 = 10 # 1er centre à choisir en fonction de calibration_image
y0 = 11

calibration_image=pre_processing(angle, pitch, file_path0, bg_path)
LFM,centerX,centerY,CX,CY, centerX_ini, centerY_ini=calibrate_LFM3(calibration_image, x0, y0)

In [ ]:
import importlib
import fourierslice_refocusing
importlib.reload(fourierslice_refocusing)

radianceS = []
sideS = []
rendered_heightS = []
rendered_widthS = []
mireS = []

for file_path in file_pathS:
    img=pre_processing(angle,pitch, file_path, bg_path)
    LFM01=compute_LFM((img/(calibration_image+1))*np.mean(calibration_image),centerX,centerY,CX,CY)
    rendered_height,rendered_width=LFM01.shape[0],LFM01.shape[1]
    side=LFM01.shape[2]                                          
    radiance=LFM01
    
    # Extract the filename without extension and add '_persp'
    original_filename = os.path.basename(file_path)
    filename_wo_ext = os.path.splitext(original_filename)[0]
    new_filename = f"{filename_wo_ext}_fourierslice.tiff"

    refocus_stack, fft_stack, Z = fourierslice_refocusing.refocus_test(radiance, min=-60, max=60, Padding=False)
    # tiff.imwrite(os.path.join(save_dir, new_filename), refocus_stack)

    radianceS.append(radiance)
    sideS.append(side)
    rendered_heightS.append(rendered_height)
    rendered_widthS.append(rendered_width)

In [ ]:
depths = [-84,-71,-58,-51,-38,-25,-12,0,26,40,46,73,79,106]
slice = [33,48,58,64,80,84,93,97,113,127,130,149,154,188]
alphas = []
zs = []
for s in slice:
    z = s * 5.18 - 505
    alpha = 1 + ( z * (33**2))/165000
    alphas.append(alpha)
    zs.append(z)
    # print("z:", z, "alpha:", alpha)

depths =np.array(depths)
zs = np.array(zs)
scale_factors = zs / depths
print(scale_factors)
print(zs)
print(alphas)

plt.figure()
plt.plot(depths, slice)
plt.title("Slice en fonction de la profondeur")
plt.grid(True)  
plt.show()

plt.figure()
plt.plot(depths, alphas)
plt.title("Alpha en fonction de la profondeur")
plt.grid(True)                    # Ajout de la grille
ax = plt.gca()  # Get current axes
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.show()

plt.figure()
plt.plot(depths, zs)
plt.title("Profondeur calculée en fonction de la profondeur réelle")
plt.grid(True)                    
ax = plt.gca()  # Get current axes
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import Polynomial
from scipy.stats import linregress

# Provided data
depths = np.array([-84, -71, -58, -51, -38, -25, -12, 0, 26, 40, 46, 73, 79, 106])
alphas = np.array([-1.2048, -0.6920, -0.3501, -0.1450, 0.4020, 0.5388, 0.8465,
                   0.9832, 1.5302, 2.0089, 2.1114, 2.7610, 2.9320, 4.0943])
z_refocus = np.array([-334.06, -256.36, -204.56, -173.48, -90.6, -69.88,
                      -23.26, -2.54, 80.34, 152.86, 168.4, 266.82, 292.72, 468.84])

# Fit a polynomial: z_refocus = f(depth)
degree = 3
coefs = Polynomial.fit(depths, z_refocus, deg=degree).convert().coef
poly_alpha_fit = Polynomial.fit(depths, alphas, deg=degree).convert()
# print("Fitted polynomial coefficients:", coefs)
slope, intercept, r_value, p_value, std_err = linregress(depths, alphas)
print(f"alpha = {slope:.4f} * depth + {intercept:.4f}")
print(f"R2 = {r_value**2:.4f}")

# Define the function
def depth_to_zrefocus(depth):
    return sum(c * depth**i for i, c in enumerate(coefs))

def depth_to_alpha(depth):
    """Return alpha given object-space depth (µm)"""
    # return poly_alpha_fit(depth)
    return 0.0251 * depth + 1.0742

# Plot fit
depths_dense = np.linspace(min(depths), max(depths), 300)
z_fit = depth_to_zrefocus(depths_dense)
alpha_fit = depth_to_alpha(depths_dense)

plt.figure()
plt.plot(depths, z_refocus, '-', label='Data')
plt.plot(depths_dense, z_fit, '-', label=f'Fit polynomial')
plt.xlabel("Profondeur réelle (µm)")
plt.ylabel("Profondeur souhaitée (µm)")
plt.grid(True)                    
ax = plt.gca()  # Get current axes
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.legend()
plt.show()

plt.figure()
plt.plot(depths, alphas, '-', label='Data')
plt.plot(depths_dense, alpha_fit, '-', label=f'Fit polynomial')
plt.xlabel("Profondeur (µm)")
plt.ylabel("Alpha")
plt.grid(True)
plt.legend()
plt.show()